# Structure Mutations & Trajectory Tracking

This notebook demonstrates how to modify crystal structures and track their evolution.

**What you'll learn:**
- Apply mutations: lattice scaling, element substitution, site operations
- Track structure evolution through a trajectory
- Export trajectories for visualization and analysis


In [1]:
from ggen import GGen


Warp DeprecationWarning: The symbol `warp.vec` will soon be removed from the public API. Use `warp.types.vector` instead.


## Generate a Starting Structure

First, create a structure to mutate. Enable trajectory tracking to record all changes:


In [2]:
ggen = GGen(enable_trajectory=True)

result = ggen.generate_crystal(
    formula="Fe4Ni4",
    num_trials=10,
    optimize_geometry=True,
    preserve_symmetry=True
)

print(f"Generated: {result['name']}")
print(f"Space group: {result['final_space_group_symbol']}")
print(f"Energy: {result['best_crystal_energy']:.4f} eV")


/Users/mmoderwell/.pyenv/versions/3.12.0/envs/materials/lib/python3.12/site-packages/orb_models/utils.py:30: UserWarning: Setting global torch default dtype to torch.float32.
  warnings.warn(f"Setting global torch default dtype to {torch_dtype}.")


Generated: Fe4Ni4 (I4_1/amd)
Space group: I4_1/amd
Energy: -57.1155 eV


## Lattice Mutations

Scale and shear the lattice:


In [3]:
# Scale lattice by 5% (expand)
ggen.scale_lattice(1.05)
print(f"After scaling: Volume = {ggen.get_structure().volume:.2f} Å³")

# Shear the lattice angles slightly
ggen.shear_lattice((1.0, -0.5, 0.5))  # Perturb alpha, beta, gamma
print(f"After shearing: {ggen.get_structure().lattice}")


After scaling: Volume = 105.22 Å³
After shearing: 3.7092323464482098 0.0 0.03236998029265939
-0.031806257325045 3.708672246806802 -0.06473749548599417
0.0 0.0 7.6472716841306285


## Element Substitution

Replace some atoms with a different element:


In [4]:
# Substitute 50% of Ni with Co
ggen.substitute("Ni", "Co", fraction=0.5)
print(f"After substitution: {ggen.get_structure().composition}")


After substitution: Fe4 Ni2 Co2


## Site Perturbations

Add small random displacements to break symmetry:


In [5]:
# Jitter all atomic positions slightly
ggen.jitter_sites(sigma=0.02)  # 0.02 Å standard deviation
print("Applied random jitter to all sites")


Applied random jitter to all sites


## Re-optimize After Mutations

After mutations, re-optimize to find a local energy minimum:


In [6]:
energy_before = ggen.calculate_energy()
print(f"Energy before optimization: {energy_before:.4f} eV")

ggen.optimize_geometry(max_steps=200, fmax=0.05)

energy_after = ggen.calculate_energy()
print(f"Energy after optimization: {energy_after:.4f} eV")
print(f"Energy change: {energy_after - energy_before:.4f} eV")


Energy before optimization: -58.1678 eV
Energy after optimization: -59.4967 eV
Energy change: -1.3290 eV


## Batch Mutations

Apply multiple mutations at once:


In [7]:
# Define a sequence of mutations
mutations = [
    {"op": "scale_lattice", "scale_factor": 0.98},
    {"op": "substitute", "element_from": "Fe", "element_to": "Mn", "fraction": 0.25},
    {"op": "jitter_sites", "sigma": 0.01}
]

ggen.mutate_crystal(mutations, repair=True)
print(f"After batch mutations: {ggen.get_structure().composition}")


After batch mutations: Fe3 Mn1 Ni2 Co2


## View the Trajectory

The trajectory records every structure modification:


In [8]:
trajectory = ggen.get_trajectory()

print(f"Trajectory has {len(trajectory)} frames:\n")
for frame in trajectory:
    print(f"  Frame {frame['frame_index']:2d}: {frame['composition']:20s}  "
          f"E={frame['energy']:.4f} eV  ({frame['operation']})")


Trajectory has 28 frames:

  Frame  0: Fe4Ni4                E=-52.2682 eV  (None)
  Frame  1: Fe4Ni4                E=-52.2682 eV  (optimize_geometry)
  Frame  2: Fe4Ni4                E=-54.5028 eV  (optimize_geometry)
  Frame  3: Fe4Ni4                E=-56.4469 eV  (optimize_geometry)
  Frame  4: Fe4Ni4                E=-56.9630 eV  (optimize_geometry)
  Frame  5: Fe4Ni4                E=-57.1154 eV  (optimize_geometry)
  Frame  6: Fe4Ni4                E=-57.1155 eV  (optimize_geometry)
  Frame  7: Fe4Ni4                E=-57.1155 eV  (optimize_geometry)
  Frame  8: Fe4Ni4                E=-57.1155 eV  (scale_lattice)
  Frame  9: Fe4Ni4                E=-56.2200 eV  (scale_lattice)
  Frame 10: Fe4Ni4                E=-56.2200 eV  (shear_lattice)
  Frame 11: Fe4Ni4                E=-56.2155 eV  (shear_lattice)
  Frame 12: Co2Fe4Ni2             E=-58.5925 eV  (None)
  Frame 13: Co2Fe4Ni2             E=-58.1678 eV  (None)
  Frame 14: Co2Fe4Ni2             E=-58.1678 eV  (optimize_geo

## Export the Trajectory

Save the trajectory for visualization in other tools:


In [9]:
# Export as ASE trajectory (can be visualized with ASE GUI or pymatviz)
traj_file = ggen.export_trajectory("evolution.traj")
print(f"Exported trajectory to: {traj_file}")


Exported trajectory to: evolution.traj


## Visualize the Trajectory

If pymatviz is installed, view the trajectory interactively:


In [11]:
try:
    from pymatviz import TrajectoryWidget
    from ase.io import read
    
    atoms = read(traj_file, index=':')
    display(TrajectoryWidget(atoms))
except ImportError:
    print("Install pymatviz for trajectory visualization: pip install pymatviz")


## Available Mutations

Here's the full list of mutation operations:

| Operation | Description | Key Parameters |
|-----------|-------------|----------------|
| `scale_lattice` | Scale lattice vectors | `scale_factor`, `isotropic` |
| `shear_lattice` | Modify lattice angles | `angle_deltas` (α, β, γ) |
| `substitute` | Replace elements | `element_from`, `element_to`, `fraction` |
| `add_site` | Add an atom | `element`, `coordinates` |
| `remove_site` | Remove atoms | `site_indices` or `element`, `max_remove` |
| `move_site` | Displace an atom | `site_index`, `displacement` |
| `jitter_sites` | Random displacements | `sigma`, `element` (optional) |
| `symmetry_break` | Break crystal symmetry | `displacement_scale`, `angle_perturbation` |
| `change_space_group` | Transform to new space group | `target_space_group` |


## Summary

In this notebook you learned how to:
- Apply individual mutations (scaling, substitution, jitter)
- Batch multiple mutations together
- Track structure evolution through trajectories
- Export and visualize trajectories

These mutation operations are useful for:
- Evolutionary/genetic algorithms for structure optimization
- Creating training data for ML models
- Exploring structural variations around a known phase
